# Nadex KPI Report Generator

**Sprint 3.5** - Generates HTML KPI Dashboard from pre-computed backtest results

**Purpose:** Load pre-computed results from S3, generate HTML dashboard only.
- NO backtesting computation
- Uses results saved by `nadex-backtesting.ipynb`

**Output:**
- `reports/kpi_dashboard.html` (local file)
- S3: `reports/<date>/kpi_dashboard.html`

In [ ]:
%pip install pandas numpy pyyaml boto3 jinja2 --quiet

In [ ]:
import sys
sys.path.append('../src')

import yaml
from datetime import datetime

# Import from nadex_common (flat structure)
from nadex_common import (
    create_s3_clients,
    BacktestResults,
    generate_html_dashboard
)

print("✓ Imports successful")

## 1. Load Pre-Computed Backtest Results from S3

These results were saved by `nadex-backtesting.ipynb`.

In [ ]:
# Load S3 configuration
with open('../configs/s3.yaml', 'r') as f:
    s3_cfg = yaml.safe_load(f)

clients = create_s3_clients(region=s3_cfg.get('region'))
s3_client = clients['private']
BUCKET = s3_cfg['bucket']

print(f"✓ Bucket: {BUCKET}")

In [ ]:
# Load latest backtest results from S3
# Use date=None to load from 'latest' folder
# Or specify date='2025-12-12' for specific date

try:
    results = BacktestResults.load_from_s3(s3_client, BUCKET)
    print(f"\n✓ Loaded: {results}")
except FileNotFoundError as e:
    print(f"\n⚠️  No backtest results found in S3!")
    print(f"   Run nadex-backtesting.ipynb first to generate and save results.")
    print(f"   Error: {e}")
    results = None

## 2. Display KPI Summary

In [ ]:
if results:
    kpis = results.kpis
    
    print("\n" + "=" * 70)
    print("📊 KPI SUMMARY (from saved backtest results)")
    print("=" * 70)
    print(f"Win Rate:        {kpis['win_rate']:.2%}")
    print(f"Total Trades:    {kpis['total_trades']:,}")
    print(f"Wins:            {kpis['wins']:,}")
    print(f"Losses:          {kpis['losses']:,}")
    print(f"")
    print(f"Gross P&L:       ${kpis['gross_pnl']:,.2f}")
    print(f"Commissions:     ${kpis['commissions']:,.2f}")
    print(f"Net P&L:         ${kpis['net_pnl']:,.2f}")
    print(f"")
    print(f"Max Drawdown:    ${kpis['max_drawdown']:,.2f}")
    print(f"Max Drawdown %:  {kpis['max_drawdown_pct']:.2f}%")
    print(f"Recovery Days:   {kpis['recovery_days']}")
    print(f"")
    print(f"Period:          {kpis['date_start'].date()} to {kpis['date_end'].date()}")
    print(f"Generated:       {results.generated_at}")
    print(f"Strategy:        {results.strategy_params}")
    print("=" * 70)

## 3. Generate HTML Dashboard

In [ ]:
if results:
    # Generate HTML from loaded KPIs
    COMMISSION_PER_CONTRACT = results.strategy_params.get('commission_rate', 1.00)
    html_content = generate_html_dashboard(kpis, commission_per_contract=COMMISSION_PER_CONTRACT)
    
    # Save locally
    local_path = '../reports/kpi_dashboard.html'
    with open(local_path, 'w') as f:
        f.write(html_content)
    
    print(f"✓ HTML dashboard saved to: {local_path}")

## 4. Upload HTML to S3

In [ ]:
if results:
    # Upload to S3
    today = datetime.now().strftime('%Y-%m-%d')
    s3_key = f"reports/{today}/kpi_dashboard.html"
    
    s3_client.put_object(
        Bucket=BUCKET,
        Key=s3_key,
        Body=html_content.encode('utf-8'),
        ContentType='text/html'
    )
    
    # Also save to reports/latest/
    latest_key = "reports/latest/kpi_dashboard.html"
    s3_client.put_object(
        Bucket=BUCKET,
        Key=latest_key,
        Body=html_content.encode('utf-8'),
        ContentType='text/html'
    )
    
    print(f"✓ Uploaded to S3: s3://{BUCKET}/{s3_key}")
    print(f"✓ Uploaded to S3: s3://{BUCKET}/{latest_key}")

## 5. Complete!

The KPI dashboard has been generated from pre-computed backtest results.

**Workflow:**
1. `nadex-backtesting.ipynb` runs backtesting, saves results to S3
2. This notebook loads those results, generates HTML dashboard

**Benefits:**
- No duplicate computation
- Fast dashboard generation (seconds vs minutes)
- Single source of truth for metrics

In [ ]:
if results:
    print("\n" + "=" * 70)
    print("✓ KPI Dashboard Generation Complete!")
    print("=" * 70)
    print(f"\nLocal files:")
    print(f"  - {local_path}")
    print(f"\nS3 files:")
    print(f"  - s3://{BUCKET}/{s3_key}")
    print(f"  - s3://{BUCKET}/{latest_key}")
    print(f"\nSource data:")
    print(f"  - s3://{BUCKET}/backtest/results/latest/")
    print("=" * 70)
else:
    print("\n⚠️  No results to display. Run nadex-backtesting.ipynb first.")